In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import gc
import re

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from itertools import cycle
from IPython.display import display
import pickle, os

import seaborn as sns

BES_data_folder = "../BES_analysis_data/"
BES_code_folder = "../BES_analysis_code/"

from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE

In [ ]:
%%time
counts = BES_numerics_only[BES_num_and_cat["wave9"]==1 ].count()
counts_by_individuals = BES_numerics_only[BES_num_and_cat["wave9"]==1 ].count(axis=1) # lets see
f, axarr = plt.subplots( 2 ,figsize=(5, 5) )
axarr[0].hist(counts,bins=50)
axarr[1].hist(counts_by_individuals,bins=50)

In [ ]:
W9_vars = [x for x in BES_numerics_only.columns if "W9" in x]


In [ ]:
# drop people who are coverage outliers
# say top and bottom 5%
# BES_red = BES_numerics_only.loc[BES_reduced["wave9"]==1 ].copy()
BES_red = BES_numerics_only.copy()
BES_red["coverage"] = BES_numerics_only.count(axis=1)/BES_numerics_only.shape[1]

In [ ]:
thresh = .00
outlier_people = BES_red[( BES_red["coverage"]>BES_red["coverage"].quantile(1-thresh) ) | 
                         ( BES_red["coverage"]<BES_red["coverage"].quantile(  thresh) ) ].index
inlier_people  = BES_red[( BES_red["coverage"]<BES_red["coverage"].quantile(1-thresh) ) & 
                         ( BES_red["coverage"]>BES_red["coverage"].quantile(  thresh) ) ].index

In [ ]:
counts = BES_numerics_only.loc[inlier_people].count() # super slow!
counts_by_individuals = BES_numerics_only.loc[inlier_people].count(axis=1) # lets see
f, axarr = plt.subplots( 2 ,figsize=(5, 5) )
axarr[0].hist(counts,bins=50)
axarr[1].hist(counts_by_individuals,bins=50)

In [ ]:
thresh = 0.11
mostly_filled_out = counts[counts> (counts.max()*thresh)].index

In [ ]:
counts = BES_numerics_only.loc[inlier_people][mostly_filled_out].count() # super slow!
counts_by_individuals = BES_numerics_only.loc[inlier_people][mostly_filled_out].count(axis=1) # lets see
f, axarr = plt.subplots( 2 ,figsize=(5, 5) )
axarr[0].hist(counts,bins=20)
axarr[1].hist(counts_by_individuals,bins=20)

In [ ]:
# BES_min = BES_numerics_only.loc[inlier_people][mostly_filled_out]
BES_min = BES_numerics_only.loc[inlier_people][mostly_filled_out]

In [ ]:
BES_min.isnull().sum().sum()/(BES_min.shape[0]*BES_min.shape[1])

In [ ]:
BES_min.shape

In [ ]:
corr_mat = BES_min.corr()
# create lower triangle - diag mask
col_vars = len( corr_mat )
df = pd.DataFrame( np.arange(col_vars*col_vars).reshape(col_vars,col_vars) )
mask = np.ones(df.shape,dtype='bool')
mask[np.triu_indices(len(df))] = False
corr_mat.mask(~mask).stack().hist(bins=100)

min_corr = []

thresh = .05
x = ( ( corr_mat>(1-thresh) ) &mask ).values.nonzero()
# z = list( zip(corr_mat.columns[x[0]],corr_mat.columns[x[1]]) )
# z
for i in range( 0,len(x[0]) ):
    a = corr_mat.columns[ x[0][i] ]
    b = corr_mat.columns[ x[1][i] ]
    c = BES_min[a].notnull().mean()
    d = BES_min[b].notnull().mean()
    print( ( a,b,corr_mat[a][b], c, d, min(c,d) ) )
    min_corr.append( min(c,d) )

x = ( ( corr_mat<(thresh-1) ) &mask ).values.nonzero()
# z = list( zip(corr_mat.columns[x[0]],corr_mat.columns[x[1]]) )
# z
for i in range( 0,len(x[0]) ):
    a = corr_mat.columns[ x[0][i] ]
    b = corr_mat.columns[ x[1][i] ]
    c = BES_min[a].notnull().mean()
    d = BES_min[b].notnull().mean()
    print( ( a,b,corr_mat[a][b], c, d, min(c,d) ) )
    min_corr.append( min(c,d) )

In [ ]:
pd.DataFrame(min_corr).hist(bins=100)

In [ ]:
opts_per_cat = BES_min.apply(lambda x: len(pd.unique(x.dropna()))).sort_values()
opts_per_cat

In [ ]:
# BES_num_and_cat[opts_per_cat[opts_per_cat==2].index].dtypes # yes - all categories
two_opt_cats = opts_per_cat[opts_per_cat==2].index
two_opt_cats

In [ ]:
many_cat_drop_list = ['profile_lea', 'profile_oslaua', 'EUMIICategory']

In [ ]:
BES_min.to_hdf(BES_data_folder+"BESW8min.hdf", "BESW8min")

In [ ]:
BES_min.shape